In [1]:
#!/usr/bin/env python3
import json
import numpy as np
import requests
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

###Create Tools for code
#create tool that can append files to a list
def append_file_to_list(file):
    list = []
    f = open(file, 'r')
    for line in f:
        line = line.strip('\n')
        line = line.split('\t')
        list.append(line)
    return list


In [2]:
#open error file for deep learning
error_data = append_file_to_list('../data/clinical_analysis_data/errors_dl.tsv')
error_pd = pd.read_csv('../data/clinical_analysis_data/errors_dl.tsv', sep='\t')
training_data = pd.read_pickle('../data/training_data.pkl')
full_data = pd.read_pickle('../data/full_sample_df.pkl')

In [4]:
#Make sure that the DoCM Variants and CIViC Variants overlap

CIViC_FN = append_file_to_list("../data/clinical_analysis_data/CIViC_variants_DL_FN.txt")
DoCM_FN = append_file_to_list("../data/clinical_analysis_data/DoCM_variants_DL_FN.txt")

overlap_FN = 0

for item in DoCM_FN:
    code = item[0:5]
    for thing in CIViC_FN:
        code2 = item[0:5]
        if code == code2:
            overlap_FN += 1
            break
            
CIViC_FP = append_file_to_list("../data/clinical_analysis_data/CIViC_variants_DL_FP.txt")
DoCM_FP = append_file_to_list("../data/clinical_analysis_data/DoCM_variants_DL_FP.txt")

overlap_FP = 0

for item in DoCM_FP:
    code = item[0:5]
    for thing in CIViC_FP:
        code2 = item[0:5]
        if code == code2:
            overlap_FP += 1
            break

In [5]:
FN_no_clinical = {}
FN_clinical = {}
for number,thing in enumerate(error_data):
    if thing[2] == 'False Negative':
        code1 = thing[7:12]
        prob1 = thing[5][0:5]
        for item in CIViC_FN[1:]:
            code2 = item[0:5]
            prob2 = item[7][0:5]
            if code1 == code2 and prob1 == prob2:
                FN_clinical[number] = thing
for number,thing in enumerate(error_data):
    if thing[2] == 'False Negative' and number not in FN_clinical:
        FN_no_clinical[number] = thing
                             
FP_no_clinical = {}
FP_clinical = {}
for number,thing in enumerate(error_data):
    if thing[2] == 'False Positive':
        code1 = thing[7:12]
        prob1 = thing[5][0:5]
        for item in CIViC_FP[1:]:
            code2 = item[0:5]
            prob2 = item[7][0:5]
            if code1 == code2 and prob1 == prob2:
                FP_clinical[number] = thing
for number,thing in enumerate(error_data):
    if thing[2] == 'False Positive' and number not in FP_clinical:
        FP_no_clinical[number] = thing

In [6]:
#Pull the non-Clinical Flase Positives and take random 30

FP_no_clinical_IGV = []
for item in range(45):
    number = np.random.choice((list(FP_no_clinical.keys())))
    FP_no_clinical_IGV.append(FP_no_clinical[number])


In [7]:
#Pull the non-Clinical Flase Negatives and take random 30

FN_no_clinical_IGV = []
for item in range(45):
    number = np.random.choice((list(FN_no_clinical.keys())))
    FN_no_clinical_IGV.append(FN_no_clinical[number])

In [8]:
#Pull the Clinical False Negatives and False Positives
FP_clinical_IGV = []
for k,v in FP_clinical.items():
    FP_clinical_IGV.append(v)
    
FN_clinical_IGV = []
for k,v in FN_clinical.items():
    FN_clinical_IGV.append(v)

In [81]:
#print all clinical to lists
np.savetxt("../data/manual_rereview/clin_FP.csv", FP_clinical_IGV, delimiter=",", fmt='%s')
np.savetxt("../data/manual_rereview/clin_FN.csv", FN_clinical_IGV, delimiter=",", fmt='%s')

In [13]:
#print all non_clinical to lists
np.savetxt("../data/non_clin_FP.csv", FP_no_clinical_IGV, delimiter=",", fmt='%s')
np.savetxt("../data/non_clin_FN.csv", FN_no_clinical_IGV, delimiter=",", fmt='%s')

In [10]:
# get full data
pd.set_option('display.max_colwidth', -1)
full_data[full_data['individual_name'] == "H_RO-WU018"]#[['subject_name','model_id', 'tissue_type', 'gms_bam_path']]

,xml_file_path,bed_file_path,track_name,xml_bam_path,model_id,model_type,subject_name,individual_name,build_id,gms_bam_path,...,sequencing_scope,project,tumor_normal_pair,has_met,has_pre_malignant,has_relapse,sequencing_context,species_name,ref-seq-build,formated_bed_file_path
1657,NaN,/gscmnt/gc2543/griffithlab/bainscou/manual_review_data/lee/BKM120/H_RO-WU018.bed,NaN,NaN,e373b3d7a28e47a890cc2dfbb4fed623,reference alignment,H_RO-WU018-WU018_BKM120,H_RO-WU018,5c34a155422c430c9990f252ec2d129b,/gscmnt/gc7002/info/build_merged_alignments/merged-alignment-blade8-1-2.gsc.wustl.edu-apipe-builder-14480-32af0e1541b445cc8283a6ddecc985ed/32af0e1541b445cc8283a6ddecc985ed.bam,...,exome,lee/BKM120,True,False,False,False,capture,human,GRCh37-lite-build37,/gscmnt/gc2543/griffithlab/bainscou/manual_review_data/lee/BKM120/H_RO-WU018.bed.formated
1674,NaN,/gscmnt/gc2543/griffithlab/bainscou/manual_review_data/lee/BKM120/H_RO-WU018.bed,NaN,NaN,f7de451f0b5f470d9095d561df7e296b,reference alignment,H_RO-WU018-gl1310144,H_RO-WU018,49411c66a4ad4085969f4758f8ca3c2f,/gscmnt/gc7002/info/build_merged_alignments/merged-alignment-blade14-2-14.gsc.wustl.edu-apipe-builder-30471-4bb11f4fe80348a3a33d4d25439c6dad/4bb11f4fe80348a3a33d4d25439c6dad.bam,...,exome,lee/BKM120,True,False,False,False,capture,human,GRCh37-lite-build37,/gscmnt/gc2543/griffithlab/bainscou/manual_review_data/lee/BKM120/H_RO-WU018.bed.formated


In [156]:
arr = []
for item in range(180):
    arr.append(item)
arr.pop(0)

print(arr)

random.shuffle(arr)
print(arr)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179]
[36, 146, 42, 40, 163, 13, 57, 35, 14, 97, 29, 148, 19, 121, 130, 54, 73, 33, 8, 134, 41, 155, 78, 43, 110, 49, 5, 16, 127, 79, 56, 20, 77, 99, 152, 162, 15, 38, 104, 31, 32, 90, 30, 23, 98, 83, 7, 107, 81, 69, 3